In [1]:
import os
import sys
sys.path.append('../../../ray8/lib/chexnet/')
os.environ['CUDA_VISIBLE_DEVICES']='7'
import time
import sys
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms
from DensenetModelsLocal import DenseNet121

from glob import glob
from tqdm import tqdm
import numpy as np

In [2]:
transCrop = 512
nnClassCount = 14
pathModel = '../../../ray8/lib/chexnet/m-15012020-084229.pth.tar'
model = DenseNet121(transCrop, nnClassCount, True)
modelCheckpoint = torch.load(pathModel)
model.load_state_dict(modelCheckpoint['state_dict'])
model = torch.nn.DataParallel(model).cuda()
model = model.module.densenet121.features
model.eval()

/home/zhangwd/.conda/envs/py36/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


kernelCount:	1024


Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

In [3]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList = []
transformList.append(transforms.Resize(transCrop))
transformList.append(transforms.ToTensor())
transformList.append(normalize)
transformSequence = transforms.Compose(transformList)

In [4]:
pathImageFile = '../../../ray8/lib/chexnet/test/00009285_000.png'
imageData = Image.open(pathImageFile).convert('RGB')
imageData = transformSequence(imageData)
imageData = imageData.unsqueeze_(0)

In [5]:
feature_map = model(torch.autograd.Variable(imageData.cuda()))

In [6]:
feature_map.shape

torch.Size([1, 1024, 16, 16])

In [7]:
def gen_feature_map(pathImageFile, outdir, transformSequence):
    imageData = Image.open(pathImageFile).convert('RGB')
#     print(imageData)
    imageData = transformSequence(imageData)
    imageData = imageData.unsqueeze_(0)
    feature_map = model(torch.autograd.Variable(imageData.cuda()))
    feature_map=feature_map.squeeze().cpu().detach().numpy()
#     print(feature_map.shape)
    outfile = os.path.join(outdir, os.path.basename(pathImageFile).split('.')[0]+'.npy')
    np.save(outfile,feature_map)

In [8]:
indir = './xray_data/data'
outdir = './xray_data/features'
os.makedirs(outdir, exist_ok=True)
infiles = glob(os.path.join(indir, '*.png'))
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList = []
transformList.append(transforms.Resize([transCrop, transCrop]))
transformList.append(transforms.ToTensor())
transformList.append(normalize)
transformSequence = transforms.Compose(transformList)
for infile in tqdm(infiles):
    gen_feature_map(infile, outdir, transformSequence)

100%|██████████| 7470/7470 [17:36<00:00,  7.07it/s]
